In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('./data/dogs_encoder_data.csv')
data.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.drop('상태',axis=1),data['상태'],test_size=0.1,random_state=42)

In [ ]:
!pip install xgboost

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score, roc_auc_score, recall_score
def get_clf_eval(y_test, pred = None, pred_proba = None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}, F1:{3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=30000, learning_rate=0.0005)
xgb_model = model.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric='logloss', eval_set=[(X_train, y_train), (X_test, y_test)], verbose=True)

In [ ]:
x_pred = xgb_model.predict(X_test)
x_pred_proba = xgb_model.predict_proba(X_test)[:,1]

In [ ]:
get_clf_eval(y_test,x_pred,x_pred_proba)

In [ ]:
import matplotlib.pyplot as plt
from xgboost import plot_importance

results = xgb_model.evals_result()
print(results)

In [ ]:
import matplotlib.pyplot as plt
from xgboost import plot_importance

results = xgb_model.evals_result()

epochs = len(results['validation_0']['logloss'])
x_axis = range(0, epochs)
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_1']['logloss'], label='Test Log Loss', color = 'b')
ax.plot(x_axis, results['validation_0']['logloss'], label='Train Log Loss', color = 'r')
ax.legend()
plt.xlabel('Epochs')
plt.ylabel('Log Loss')
plt.title('XGBoost Log Loss')
plt.show()

In [ ]:
import pickle
file_path = './model/dog_adopt_classifier.model'
pickle.dump(xgb_model, open(file_path, 'wb'))